<a href="https://colab.research.google.com/github/Nouh-essam/DEPI.NOOH/blob/main/Recommendation_System_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np # linear algebra
import pandas as pd

In [ ]:
#loading files via numpy
from numpy import loadtxt
def load_ratings_small():
  file = open('Apartment_Y.csv', 'rb')
  Y = loadtxt(file,delimiter = ",")

  file = open('Apartment_R.csv', 'rb')
  R = loadtxt(file,delimiter = ",")
  return(Y,R)

In [ ]:
Y, R = load_ratings_small()

In [ ]:
apartmentList_df=pd.read_csv('Apartment Database.csv',index_col=0)

In [ ]:
num_apartments , num_users = Y.shape
my_ratings = np.zeros(num_apartments)


In [ ]:
Y,R=load_ratings_small()
Y=np.c_[my_ratings,Y]
R=np.c_[(my_ratings!=0).astype(int),R]

In [ ]:
#Normalize
Y_mean=(np.sum(Y*R,axis=1)/(np.sum(R,axis=1)+1e-12)).reshape(-1,1)
Y_norm=Y-Y_mean*R

In [ ]:
num_apartments,num_users=Y.shape
num_features=11

tf.random.set_seed(1234)
W=tf.Variable(tf.random.normal((num_users,num_features),dtype=tf.float64),name='W')
X=tf.Variable(tf.random.normal((num_apartments,num_features),dtype=tf.float64),name='X')
b=tf.Variable(tf.random.normal((1,num_users),dtype=tf.float64),name='b')

optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1)

In [ ]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
  J = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R      #matmul works different for dimensions>2
  J = 0.5 * tf.reduce_sum(J**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))

  return J

In [ ]:
#  Using tensorflow , we can find derivative of cost function
iterations=700
lambda_=1
for i in range(iterations):
  with tf.GradientTape() as tape:
    # Compute the cost (forward pass included in cost)
    cost=cofi_cost_func(X,W,b,Y_norm,R,lambda_)
  grad=tape.gradient(cost,[X,W,b])
  # Run one step of gradient descent by updating
  # the value of the variables to minimize the loss.
  optimizer.apply_gradients(zip(grad,[X,W,b]))
  # Log periodically.
  #if i % 200 == 0:
    #print(f"Training loss at iteration {i}: {cost:0.1f}")

In [ ]:
p=np.matmul(X.numpy(),tf.transpose(W.numpy())) + b.numpy()

In [ ]:
pm=p+Y_mean

In [ ]:
my_predictions=pm[:,0]
sorted_index=tf.argsort(my_predictions,direction='DESCENDING')

In [ ]:
apartmentList_df = apartmentList_df.iloc[:len(my_predictions), :]
apartmentList_df['pred']=my_predictions
#apartmentList_df

<ipython-input-15-db3e8c051ba9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apartmentList_df['pred']=my_predictions


In [ ]:
apartmentList_df=apartmentList_df.reindex(columns=["pred", "Mean rating","Area","Price","Level of Luxury","floor","Elevator","Finishing","Number of Rooms",
                                           "Ability of Negotiation","Overlooks"])
#apartmentList_df

In [ ]:
# Assuming 'my_predictions' is the column you want to sort
sorted_indices = np.argsort(my_predictions, kind='stable')  # Use numpy's argsort
apartmentList_df = apartmentList_df.iloc[sorted_indices].sort_values('Mean rating', ascending=False)
apartmentList_df.iloc[:99]

,pred,Mean rating,Area,Price,Level of Luxury,floor,Elevator,Finishing,Number of Rooms,Ability of Negotiation,Overlooks
Apartment Number,,,,,,,,,,,
45.0,3.718059,2.980392,170.0,"4,750,000",very good,5.0,yes,delux,3.0,yes,sea and main street
43.0,3.693059,2.960784,275.0,"8,000,000",very good,7.0,yes,delux,3.0,no,sea and tourist walk
44.0,3.454645,2.843137,200.0,"4,500,000",very good,9.0,yes,super lux,3.0,yes,main street and garden
47.0,3.357084,2.764706,185.0,"5,000,000",very good,11.0,no,delux,2.0,no,sea and main street
52.0,3.150618,2.725490,100.0,"1,750,000",very good,6.0,yes,super lux,2.0,yes,none
...,...,...,...,...,...,...,...,...,...,...,...
61.0,2.259523,1.882353,120.0,"1,650,000",good,9.0,no,super lux,3.0,yes,none
17.0,2.712177,1.862745,145.0,"2,700,000",normal,4.0,no,lux,3.0,no,none
38.0,2.632345,1.862745,65.0,"350,000",normal,4.0,no,full,2.0,yes,none


In [ ]:
apartmentList_df.to_csv('output.csv', index=False)


In [ ]:
from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>